<a href="https://colab.research.google.com/github/RudyMartin/dsai-2024/blob/main/tf_transfer_learning_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transfer Learning Using MobileNetV2**


**What this script does**

This script trains an AI program to recognize images of rock, paper, and scissors using a method called transfer learning, where it adapts a pre-trained model, MobileNetV2, for this specific task. It processes images to ensure uniformity, enhances them for better learning, and automatically adjusts the training process to achieve the best performance. After training, it evaluates the model's accuracy using new images and provides a visual report showing how well the model performs. Finally, it saves the trained model and displays graphs of the model's learning progress over time. This approach helps the AI learn efficiently and effectively from the images.

**Requirements**

The images for rock, paper, and scissors are uploaded into seperate folders

*   Train (80%)
*   Test (20%)

Note the batch size of 32, so this may throw and error on small sets of pictures.

In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from google.colab import drive


In [ ]:
# Record the start time for performance evaluation
start_time = datetime.datetime.now()

# Initialize Google Drive for storing and accessing data directly
drive.mount('/content/gdrive')
root_dir = '/content/gdrive/My Drive/'
rps_dir = os.path.join(root_dir, 'rps')
train_dir = os.path.join(rps_dir, 'train')
test_dir = os.path.join(rps_dir, 'test')
model_dir = os.path.join(root_dir, 'model')
os.makedirs(model_dir, exist_ok=True)

# Ensure the data directory exists
if not os.path.exists(rps_dir):
    raise FileNotFoundError(f"Directory {rps_dir} does not exist.")
print(f"'rps' directory contents: {os.listdir(rps_dir)}")

In [ ]:
# Define a custom preprocessing function to ensure uniform image sizes and shapes
def make_square_and_resize(image):
    """Pad an image to make it square and resize it to (224, 224)."""
    target_size = (224, 224)
    height, width = image.shape[:2]
    delta_w = max(height - width, 0)
    delta_h = max(width - height, 0)
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    color = [255, 255, 255]  # white background for padding
    new_img = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    new_img = cv2.resize(new_img, target_size)
    return new_img

# Prepare image data generators with real-time augmentation and custom preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=make_square_and_resize
)
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=make_square_and_resize)

# Load images from directories and prepare them for training and validation
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')


In [ ]:
# Setup the CNN model using MobileNetV2 as a base model for transfer learning
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the convolutional base
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions, name='RPS_MobileNetV2')
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

# Configure callbacks for saving the best model and early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(os.path.join(model_dir, 'best_model.h5'), save_best_only=True)
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[early_stopping, model_checkpoint],
    verbose=2
)

In [ ]:
# Evaluate the model using test data and generate detailed classification metrics
def evaluate_model(generator, model):
    all_labels = []
    all_preds = []
    for images, labels in generator:
        preds = model.predict(images)
        all_preds.extend(preds)
        all_labels.extend(labels)
        if len(all_labels) >= len(generator.labels):  # Prevent infinite loop
            break
    y_pred = np.argmax(np.array(all_preds), axis=1)
    y_true = np.argmax(np.array(all_labels), axis=1)
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=generator.class_indices.keys(), yticklabels=generator.class_indices.keys())
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=generator.class_indices.keys()))

evaluate_model(test_generator, model)

In [ ]:
# Save the trained model for later use or deployment
model.save(os.path.join(model_dir, model.name), save_format='tf')

In [ ]:
# Visualize the training history to understand the learning trend
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], 'bo-', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'ro-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'bo-', label='Training Loss')
plt.plot(history.history['val_loss'], 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

# Calculate and print the total execution time
end_time = datetime.datetime.now()
duration = end_time - start_time
print(f"Execution time: {str(duration)}")